In [180]:
###import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
from scipy import optimize
from sklearn.linear_model import LinearRegression
import seaborn as sns
from datetime import timedelta

In [181]:
#read file
Blitz = pd.read_csv('TempBlitzFormData.csv')
Blitz.dropna(subset=['Start time (hh:mm AM/PM)', 'End time (hh:mm AM/PM)'], inplace=True) #drop null time inputs

def time_sanitation(t):
    ###Standardizes user-input times###
    t = t.str.replace('AM|PM|am|pm|:|;', '', regex=True).str.strip().astype(int) #strips input to numbers
    
    standardization = []
    for x in t: 
        if x < 100: standardization.append(x*100) #change all values such as 12 to 1200
        if x >= 100 and x < 1299: standardization.append(x) #keep all values in standard 12hr time
        if x > 1299: standardization.append(x-1200) #change all military times to 12hr
    t = standardization
    
    length = []
    for x in t: 
        if len(str(x)) < 4: length.append(f'0{x}') #if 3 digit number, add 0 in front of value (eg 134 to 0134)
        else: length.append(x)
    t = length
    
    to_time = []    
    for x in t:
        x = pd.to_datetime(x, format='%H%M') #returns 1900-01-01 xx:xx:xx
        y = x.time() #remove placehoder dates, keeps time formatting
        to_time.append(y)    
    t = to_time
    ampm = []
    for x in t: #appends AM or PM based on input time
        if x.hour > 7 and x.hour < 12: ampm.append(f'{x} AM')
        if x.hour < 7 or x.hour >= 12: ampm.append(f'{x} PM')
    t = ampm
    
    return t

Blitz['Start time (hh:mm AM/PM)'] = time_sanitation(Blitz['Start time (hh:mm AM/PM)'])
Blitz['End time (hh:mm AM/PM)'] = time_sanitation(Blitz['End time (hh:mm AM/PM)'])

#single end time value was input as 12:37 instead of 2:37. 2:37 matches 2:32 start time and 15:35 Timestamp
Blitz.loc[86, 'End time (hh:mm AM/PM)'] = '2:37:00'

#single start time value was input as 5:57 instead of 15:57. 2:57 matches 3:02 end time and 16:02 Timestamp
Blitz.loc[108, 'Start time (hh:mm AM/PM)'] = '2:57:00'

###to remove "practice" values prior to August:

#remove timezone from Timestamp so that it can be read as a DateTime
Blitz['Timestamp'] = Blitz['Timestamp'].str.replace('MDT', '', regex=True)
#extract month from DateTime
Blitz['Month'] = pd.DatetimeIndex(Blitz['Timestamp']).month
#ignoring dates not part of August blitz - not sure if these are needed
AugBlitz = Blitz[Blitz['Month'] == 8].reset_index(drop=True)

In [182]:
###Standardizing and cleaning pole and logger #s
###values cross-referenced with HoboLocations_PoleLocations.csv (referred to as HL_PL.csv)

AugBlitz['Utility Pole #'] = AugBlitz['Utility Pole #'].str.upper() #standardized capitalization
AugBlitz['Utility Pole Temperature Logger # '] = AugBlitz['Utility Pole Temperature Logger # '].str.upper()
AugBlitz['Utility Pole #'] = AugBlitz['Utility Pole #'].str.replace('TO','TP') #Corrected pole # typo
AugBlitz['Utility Pole Temperature Logger # '] = AugBlitz['Utility Pole Temperature Logger # '].str.replace('B7','BT') #Corrected pole # typo
AugBlitz['Utility Pole Temperature Logger # '] = AugBlitz['Utility Pole Temperature Logger # '].str.replace('B1','BT') 

AugBlitz.loc[[38, 79], 'Utility Pole #'] = 'CP10'                 #no visible number; collaborated locations with and standardized to HL_PL.csv
AugBlitz.loc[[85, 9, 32], 'Utility Pole #'] = 'NO NUM'            #no pole num; collaborated locations with and standardized to HL_PL.csv
AugBlitz.loc[[33], 'Utility Pole #'] = 'TPU927'                   #pole number not listed. times and logger match. index 83 notes collaborate this
AugBlitz.loc[[6], 'Utility Pole #'] = 'TP24408'                   #minor typo: TP254408 input. TP24408 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[22], 'Utility Pole #'] = 'TP24199'                  #minor typo: TP2499 input. TP24199 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[86], 'Utility Pole #'] = 'TP3890'                   #minor typo: TP3800 input. TP3890 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[83], 'Utility Pole #'] = 'TPU927'                   #minor typo: TPU827 input. TPU927 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[54], 'Utility Pole #'] = 'TP6340'                   #minor typo: TP 6340 input. TP6340 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[93, 25], 'Utility Pole #'] = 'TP21897/A1353217'     #minor typo on #93: TP21879 input. TP21897 matches pole/ logger number in HL_PL.csv 
                                                                  #Standardized both to HL_PL.csv
AugBlitz.loc[[23, 72, 47], 'Utility Pole #'] = '52/1'             #standardized pole number to HL_PL.csv
AugBlitz.loc[[77, 81, 27], 'Utility Pole #'] = 'TP21631/A1353441' #standardized pole number to HL_PL.csv
AugBlitz.loc[[18], 'Utility Pole #'] = 'TP21374'                  #standardized pole number to HL_PL.csv
AugBlitz.loc[[97, 53], 'Utility Pole #'] = 'TP32711/A0572122'     #standardized pole number to HL_PL.csv
AugBlitz.loc[[11, 108], 'Utility Pole #'] = 'TP25239/A1353202'    #standardized Pole #s. Likely typo on HL_PL.csv: input value BT2130297 should be BT21302971
AugBlitz.loc[[26], 'Utility Pole #'] = 'TP21629'                  #standardized Pole #s. Likely typo in HL_PL.csv: input value BT2130251 should be BT21302951
AugBlitz.loc[[20, 68, 55], 'Utility Pole #'] = '112049 50/3'      #standardized pole #s. Likely typo in HL_PL.csv: input value 11049   50/3 should be 112049 50/3

AugBlitz.loc[[103], 'Utility Pole Temperature Logger # '] = '21223124'      #minor typo: 2122324 input. 21223124 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[3], 'Utility Pole Temperature Logger # '] = '21223121'        #minor typo: 211223121 input. 21223121 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[105], 'Utility Pole Temperature Logger # '] = '21223101'      #minor typo: 2122301 was input. 21223101 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[21], 'Utility Pole Temperature Logger # '] = '21223136'       #minor typo: 12236136 was input. 21223136 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[74], 'Utility Pole Temperature Logger # '] = '21223129'       #minor typo: 2122329 was input. 21223129 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[89], 'Utility Pole Temperature Logger # '] = '21223116'       #minor typo: 2122316 was input. 21223116 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[101], 'Utility Pole Temperature Logger # '] = 'BT21302953'    #minor typo: 21302953 was input. BT21302953 matches pole/ logger number in HL_PL.csv
AugBlitz.loc[[29, 0], 'Utility Pole Temperature Logger # '] = '21223116/BT21302967'      #standardized logger numbers to HL_PL.csv
AugBlitz.loc[[44, 12], 'Utility Pole Temperature Logger # '] = '2401795/BT21302957'      #standardized logger numbers to HL_PL.csv
AugBlitz.loc[[5, 35, 95], 'Utility Pole Temperature Logger # '] = '21162467/BT21302947'  #minor typo on #5: logger #21302954 was input. BT21302947 matches pole/ logger number in HL_PL.csv
                                                                                         #Standardized logger numbers to HL_PL.csv
AugBlitz.loc[[36, 66, 98], 'Utility Pole Temperature Logger # '] = '21223140/BT21202974' #36 has a matching pole # with 66 and 98, but different logger #
                                                                                         #probable typo between HL_PL.csv value BT21302974 and recorded BT21202973
                                                                                         #logger conditions do not match (soil vs grass) and times are pretty close, but likely the same value
                                                                                         #standardized all 3 values

#index 24 and 87 both have Logger # BT21302966, but different Pole #s. 87 Matches HL_PL.csv
#HL_PL.csv has the same Hobo_SN for pole TP6335 and TP6325. Not sure how to label index 107. 107 also has no matching logger #
#84 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#105 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#46 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#78 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#18 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#74 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#89 has Pole # and Logger # matching HL_PL.csv, but no other matches.
#34 pole # TP10504 has no matching pole in Blitz data or HL_PL.csv. Logger # 21223129 matches for pole # A1341128 in HL_PL.CV

In [183]:
AugBlitzGroups = AugBlitz.groupby(['Utility Pole #', 'Utility Pole Temperature Logger # '])
AugBlitzGroups.apply(display)

AugBlitz.to_csv(r'C:\Users\johnd\Desktop\Data Stuffs\GRIT City Temp Blitz\TempBlitzFormData.csv')

,Timestamp,Username,Your Temperature Logger #,Your Name,Start time (hh:mm AM/PM),Your Temperature Logger Location,Your temperature logger is located in:,Utility Pole #,Utility Pole Temperature Logger #,End time (hh:mm AM/PM),Any other relevant notes?,Month
20,2022/08/11 10:28:33 AM,ailene_kane@me.com,21302965,Ailene,09:25:00 AM,Sidewalk (Paved Surface),Sun,112049 50/3,21223130,09:31:00 AM,NaN,8
55,2022/08/11 1:13:51 PM,stephanie.passantino@gmail.com,21302949,Steph Schmid,12:08:00 PM,Sidewalk (Paved Surface),Sun,112049 50/3,21223130,12:13:00 PM,NaN,8
68,2022/08/11 3:34:59 PM,courtney.baxter@tnc.org,21302973,Courtney Baxter,02:29:00 PM,Sidewalk (Paved Surface),Sun,112049 50/3,21223130,02:34:00 PM,NaN,8


,Timestamp,Username,Your Temperature Logger #,Your Name,Start time (hh:mm AM/PM),Your Temperature Logger Location,Your temperature logger is located in:,Utility Pole #,Utility Pole Temperature Logger #,End time (hh:mm AM/PM),Any other relevant notes?,Month
45,2022/08/11 1:04:59 PM,courtney.baxter@tnc.org,21302958,Courtney Baxter,11:59:00 AM,Sidewalk (Paved Surface),Shade,14998,21223110,12:04:00 PM,NaN,8
91,2022/08/11 4:02:54 PM,stephanie.passantino@gmail.com,21302945,Steph Schmid,2:57:00,Sidewalk (Paved Surface),Sun,14998,21223110,03:02:00 PM,NaN,8


,Timestamp,Username,Your Temperature Logger #,Your Name,Start time (hh:mm AM/PM),Your Temperature Logger Location,Your temperature logger is located in:,Utility Pole #,Utility Pole Temperature Logger #,End time (hh:mm AM/PM),Any other relevant notes?,Month
23,2022/08/11 10:34:29 AM,ailene_kane@me.com,21302965,Ailene,09:33:00 AM,Sidewalk (Paved Surface),Shade,52/1,21223103,09:38:00 AM,NaN,8
47,2022/08/11 1:06:17 PM,stephanie.passantino@gmail.com,21302949,Steph Schmid,12:01:00 PM,Sidewalk (Paved Surface),Shade,52/1,21223103,12:06:00 PM,NaN,8
72,2022/08/11 3:42:01 PM,courtney.baxter@tnc.org,21302973,Courtney Baxter,02:36:00 PM,Sidewalk (Paved Surface),Sun,52/1,21223103,02:41:00 PM,NaN,8


,Timestamp,Username,Your Temperature Logger #,Your Name,Start time (hh:mm AM/PM),Your Temperature Logger Location,Your temperature logger is located in:,Utility Pole #,Utility Pole Temperature Logger #,End time (hh:mm AM/PM),Any other relevant notes?,Month
89,2022/08/11 4:01:32 PM,eugenejulius@gmail.com,21302954,Penny,02:59:00 PM,Grass,Sun,A1340985,21223116,03:04:00 PM,NaN,8


,Timestamp,Username,Your Temperature Logger #,Your Name,Start time (hh:mm AM/PM),Your Temperature Logger Location,Your temperature logger is located in:,Utility Pole #,Utility Pole Temperature Logger #,End time (hh:mm AM/PM),Any other relevant notes?,Month
74,2022/08/11 3:47:08 PM,stephanie.passantino@gmail.com,21302945,Steph Schmid,02:42:00 PM,Sidewalk (Paved Surface),Sun,A1341128,21223129,02:47:00 PM,NaN,8


...

,Timestamp,Username,Your Temperature Logger #,Your Name,Start time (hh:mm AM/PM),Your Temperature Logger Location,Your temperature logger is located in:,Utility Pole #,Utility Pole Temperature Logger #,End time (hh:mm AM/PM),Any other relevant notes?,Month
1,2022/08/11 9:56:14 AM,rhebert@cityoftacoma.org,21302954,Ryan and Zoee,08:51:00 AM,Grass,Sun,TP6340,BT21302950,08:56:00 AM,"Main arterial, trees on a slope across the st...",8
54,2022/08/11 1:13:13 PM,joshua.rubenstein@tnc.org,Mx2201 21302954,"Joshua Rubenstein, Austin Gobel",12:11:00 PM,Grass,Sun,TP6340,BT21302950,12:16:00 PM,NaN,8
90,2022/08/11 4:02:47 PM,acook2@cityoftacoma.org,21302954,Allison Cook,03:00:00 PM,Grass,Sun,TP6340,BT21302950,03:05:00 PM,NaN,8


,Timestamp,Username,Your Temperature Logger #,Your Name,Start time (hh:mm AM/PM),Your Temperature Logger Location,Your temperature logger is located in:,Utility Pole #,Utility Pole Temperature Logger #,End time (hh:mm AM/PM),Any other relevant notes?,Month
15,2022/08/11 10:18:39 AM,michael.yadrick@gmail.com,MX2201 21302978,Michael Yadrick,09:13:00 AM,Sidewalk (Paved Surface),Sun,TP9179,21223119,09:18:00 AM,NaN,8
50,2022/08/11 1:09:28 PM,emily.heber@tnc.org,21302965,Emily heber,12:04:00 PM,Sidewalk (Paved Surface),Shade,TP9179,21223119,12:09:00 PM,NaN,8
88,2022/08/11 4:01:04 PM,james.hong@tnc.org,21302958,James and hannah,02:55:00 PM,Sidewalk (Paved Surface),Sun,TP9179,21223119,03:01:00 PM,NaN,8


,Timestamp,Username,Your Temperature Logger #,Your Name,Start time (hh:mm AM/PM),Your Temperature Logger Location,Your temperature logger is located in:,Utility Pole #,Utility Pole Temperature Logger #,End time (hh:mm AM/PM),Any other relevant notes?,Month
19,2022/08/11 10:25:32 AM,michael.yadrick@gmail.com,MX2201 21302978,Michael Yadrick,09:19:00 AM,Sidewalk (Paved Surface),Sun,TP9182,21223111,09:25:00 AM,NaN,8
58,2022/08/11 1:18:14 PM,emily.heber@tnc.org,21302965,Emily Heber,12:12:00 PM,Sidewalk (Paved Surface),Shade,TP9182,21223111,12:18:00 PM,"Temperature logger started in shade, but the s...",8
94,2022/08/11 4:08:29 PM,james.hong@tnc.org,21302958,James and hannah,03:03:00 PM,Sidewalk (Paved Surface),Sun,TP9182,21223111,03:08:00 PM,NaN,8


,Timestamp,Username,Your Temperature Logger #,Your Name,Start time (hh:mm AM/PM),Your Temperature Logger Location,Your temperature logger is located in:,Utility Pole #,Utility Pole Temperature Logger #,End time (hh:mm AM/PM),Any other relevant notes?,Month
84,2022/08/11 3:56:54 PM,luiszepeda.chavez@gmail.com,21302965,Luis & Josh,02:55:00 PM,Grass,Sun,TP9241,21223113,03:00:00 PM,NaN,8


,Timestamp,Username,Your Temperature Logger #,Your Name,Start time (hh:mm AM/PM),Your Temperature Logger Location,Your temperature logger is located in:,Utility Pole #,Utility Pole Temperature Logger #,End time (hh:mm AM/PM),Any other relevant notes?,Month
2,2022/08/11 9:56:21 AM,emily.heber@tnc.org,21302958,Emily,08:50:00 AM,Grass,Shade,TPU927,21223106,08:56:00 AM,Edge of grass and sidewalk,8
33,2022/08/11 12:49:31 PM,james.hong@tnc.org,21302973,James and Olivia,11:44:00 AM,Grass,Sun,TPU927,21223106,11:49:00 AM,NaN,8
83,2022/08/11 3:56:51 PM,eugenejulius@gmail.com,21302954,Penny,02:52:00 PM,Sidewalk (Paved Surface),Shade,TPU927,21223106,02:57:00 PM,"The pole did not have a number, but had a smal...",8
